Analysis pinned to Kernel version 5.10

In [1]:
#todo: make the regexes more strict.

In [2]:
import re
import itertools
from itertools import islice
from subprocess import run

from pathlib import Path
import sqlite3

KERNEL = Path(".")
OUTDIR = Path("../function_survey/output/")
all_calls = OUTDIR/"cscope_all_calls.txt"
kernel_tags = OUTDIR/"kernel_tags"
test_targets = OUTDIR/"cscope_test_targets"
all_c_code = OUTDIR/"all_c_code.txt"
blame_files = Path("../blame")


connection = sqlite3.connect(OUTDIR/"function_survey.db")
cursor = connection.cursor()

In [3]:
def parse(filename, expression):
    with open(filename) as f:
        return [re.match(expression, line) for line in f]
    
def head(iterable, n=10):
    return list(islice(iterable, n))

## Ctags

In [4]:
def reset_ctags_db():
    cursor.execute("CREATE TABLE IF NOT EXISTS ctags (file, name, line, token_type, text)")
    cursor.execute("DELETE FROM ctags")
    
def parse_ctags(filename):
    expression = r"(?P<name>^[^ ]+)\s+(?P<token_type>[^ ]+)\s+(?P<linenum>\d+) (?P<path>[^ ]+) (?P<text>.*)"
    return parse(filename, expression)

In [5]:
# List all c and header files
!find -name *.c -o -name *.h > {all_c_code}
# get all tokens in the kernel.
!ctags -x -L {all_c_code} > {kernel_tags}

In [6]:
# sort ctags output by line number, give contents of each function. (C does not allow nested functions)
# assume one function per line.

#the --languages=C flag causes headers to be excluded by ctags
#running ctags with no settings results in python files being included

In [7]:
reset_ctags_db()
cursor.executemany("INSERT INTO ctags VALUES (?,?,?,?,?)", (
    (m["path"][len("./"):],
     m["name"], m["linenum"], m["token_type"], m["text"])
    for m in parse_ctags(kernel_tags)))

In [8]:
head(cursor.execute("SELECT * FROM ctags WHERE token_type = 'function'"))

[('drivers/gpu/drm/msm/adreno/a2xx.xml.h',
  'A2XX_A220_VSC_BIN_SIZE_HEIGHT',
  '1419',
  'function',
  'static inline uint32_t A2XX_A220_VSC_BIN_SIZE_HEIGHT(uint32_t val)'),
 ('drivers/gpu/drm/msm/adreno/a2xx.xml.h',
  'A2XX_A220_VSC_BIN_SIZE_WIDTH',
  '1413',
  'function',
  'static inline uint32_t A2XX_A220_VSC_BIN_SIZE_WIDTH(uint32_t val)'),
 ('drivers/gpu/drm/msm/adreno/a2xx.xml.h',
  'A2XX_CLEAR_COLOR_ALPHA',
  '2397',
  'function',
  'static inline uint32_t A2XX_CLEAR_COLOR_ALPHA(uint32_t val)'),
 ('drivers/gpu/drm/msm/adreno/a2xx.xml.h',
  'A2XX_CLEAR_COLOR_BLUE',
  '2391',
  'function',
  'static inline uint32_t A2XX_CLEAR_COLOR_BLUE(uint32_t val)'),
 ('drivers/gpu/drm/msm/adreno/a2xx.xml.h',
  'A2XX_CLEAR_COLOR_GREEN',
  '2385',
  'function',
  'static inline uint32_t A2XX_CLEAR_COLOR_GREEN(uint32_t val)'),
 ('drivers/gpu/drm/msm/adreno/a2xx.xml.h',
  'A2XX_CLEAR_COLOR_RED',
  '2379',
  'function',
  'static inline uint32_t A2XX_CLEAR_COLOR_RED(uint32_t val)'),
 ('drivers/gpu

In [9]:
# files = {}
# test_functions = set()
# for m in parse_ctags(kernel_tags):
#     # name token_type linenum path text
#     files[m["path"]] = files.get(m["path"],[]) + [m]
#     if m["token_type"] == "function" and "test" in m["name"]:
#         test_functions.add(m)

In [10]:
# {k: 
#  [(int(m["linenum"]), m["token_type"],
#    m["name"], m["text"])
#   for m in sorted(v, key=lambda m:int(m["linenum"]))]
#  for k,v in islice(files.items(), 1, 2)}

In [11]:
# [(int(m["linenum"]), m["token_type"],
#    m["name"], m["text"])
#   for m in sorted(files["./fs/9p/fid.c"],
#                   key=lambda m:int(m["linenum"]))]

In [12]:
# !grep -v variable {kernel_tags} | head -n 50 

## Blame-Parser

In [13]:
def strict_match(pattern, line, flags=0):
    m = re.match(pattern, line, flags=0)
    if m:
        return m
    raise(ValueError, f"{pattern} did not match {line}")

def blame_contents(match):
    return match["contents"].split("|")

def parse_blame_file(filename):
    expression = (r"^(?P<hash>\w{40});"
                  r"(?P<previous_file_name>[^\s;]*);"
                  r"\t(?P<contents>.*)$"
                 )
    with open(filename) as f:
        yield from (blame_contents(strict_match(expression, line)) for line in f)

In [14]:
def parse_file(filename):
    functions = {}
    state = "Not in function" # switch to enum
    for i,contents in enumerate(
        parse_blame_file(filename)
    ):
        if state == "Not in function":
            if contents[0] == "begin_function":
                contained_names = []
                called_functions = []
                function_name = None
                state = "Function declaration"
            # else: ignore current line
        elif state == "Function declaration":
            if (contents[0] == "DECL"
                and contents[1] == "function"
               ):
                function_name = contents[2].split()[0]
                state = "Function declaration breakdown"
            # else: some functions have specifiers/names on the line(s) before the declaration
            # for example find_pa in arch/alpha/boot/bootp.c, line 41
        elif state == "Function declaration breakdown":
            if contents[0] == "block":
                state = "Function Body"
            # else: ignore specifiers/arguments
        elif state == "Function Body":
            if contents[0] == "end_function":
                functions[function_name] = called_functions#, contained_names
                state = "Not in function"
            elif contents[0] == "name":
                contained_names.append(contents[1])
                current_name = contents[1]
                state = "Identifying Name"
            # else: ignore anything that is not a call/end of function
        elif state == "Identifying Name":
            # this state is the most likely to be incorrect.
            # TODO: figure out what can be after a name token
            if contents[0] == "argument_list" and contents[1] == "(": #this will probably break on no arg functions
                # assume that the name is a function
                called_functions.append(current_name)
                current_name = None
                state = "Function Body"
            elif contents[0] == "name":
                contained_names.append(contents[1])
                current_name = contents[1]
                state = "Identifying Name"
            else:
                # assume name is not a function
                current_name = None
                state = "Function Body"
        else:
            assert False, f"invalid state {state}"
        
    return functions

def parse_kernel():
    kernel_files = {}
    for filename in blame_files.rglob("*.c.blame"):
        kernel_files[filename] = parse_file(filename)
    return kernel_files

In [15]:
# How does cregit handle ifdefs in functions?

In [24]:
# file (path), item_type (function/include/ifdef/etc), name

# Call table
# caller_id, callee_name

# Functions table
# function_id, function_name

# file contents
# file, item_type, item_id


def parse_whole(filename):
    lines = parse_blame_file(filename)
        
    for start, *rest in lines:
        if (start in ("begin_unit", "end_unit", "")):
            pass
        elif start.startswith("begin_"):
            item = start[len("begin_"):]
            if item == "function":
                yield ("function", *parse_function(lines_in_item(lines, item)))
            elif item == "include":
                yield ("include", *parse_include(lines_in_item(lines, item)))
            else:
                yield skip(lines, item)

def parse_function(lines):
    lines = iter(lines)
    function_name, specifiers = parse_function_decl(lines)
    callees = parse_function_body(lines)
    return (function_name, specifiers, callees)

def parse_function_decl(lines):
    specifiers = []
    function_name = None
    for start, *rest in lines:
        if start == "specifier":
            assert len(rest) == 1
            specifiers += rest
            # are there other specifiers than static
        elif start == "DECL":
            assert rest[0] == "function", (start, rest)
            function_name = rest[1].split()[0]
        elif start == "name":
            pass # we may be able to weed out the function name, and get the types if that is usefull
        elif start == "parameter_list" and rest == [")"]:
            break  # this marks the end of the function header
        else:
            pass # TODO: check what other declaration parts end up here
    assert function_name is not None
    return function_name, specifiers

def parse_function_body(lines):
    names = []
    callees = []
    
    contents = next(lines)
    # check if this holds for empty blocks
    assert (contents == ["block", "{"]), "function body must start with block"
    
    prev_name = None
    for start, *rest in lines:
        if start == "name":
            assert len(rest) == 1
            names += rest
            prev_name = rest[0]
        elif start == "argument_list":
            if rest in [["("], ["()"]]:
                callees.append(prev_name)
            prev_name = None
        else:
            prev_name = None # assume that argument lists always follow function names directly
    return callees
            
def parse_include(lines):
    assert len(lines) == 3, lines
    lines = iter(lines)
    assert next(lines) == ["include", "#"]
    assert next(lines) == ["directive", "include"]
    start, *rest = next(lines)
    assert start == "file" and len(rest) == 1
    return rest
    
def skip(lines, item):
    """Skips to the end of a begin/end pair"""
    lines_in_item(lines, item)
    return (item, "skipped")
        
def lines_in_item(lines, item):
    """Returns a list of all lines between the begin_? and end_? markers"""
    result = []
    
    start, *rest = contents = next(lines)
    while start != f"end_{item}":
        assert not start.startswith("end_"), "end of different item found"
        assert not start.startswith("begin_"), "start of different item found"
        result.append(contents)
        start, *rest = contents = next(lines)
    return result
            

def parse_kernel_2():
    for filename in blame_files.rglob("*.c.blame"):
        yield filename, parse_file(filename)

In [23]:
list(parse_whole(
    blame_files/"arch/alpha/kernel/bugs.c.blame")
)

[('include', '<asm/hwrpb.h>'),
 ('include', '<linux/device.h>'),
 ('ifdef', 'skipped'),
 ('function', 'cpu_is_ev6_or_later', ['static'], []),
 ('function',
  'cpu_show_meltdown',
  [],
  ['cpu_is_ev6_or_later', 'sprintf', 'sprintf']),
 ('function',
  'cpu_show_spectre_v1',
  [],
  ['cpu_is_ev6_or_later', 'sprintf', 'sprintf']),
 ('function',
  'cpu_show_spectre_v2',
  [],
  ['cpu_is_ev6_or_later', 'sprintf', 'sprintf']),
 ('endif', 'skipped')]

In [34]:
kernel_files_2 = dict(parse_kernel_2())
dict(islice(kernel_files.items(), 10))

{PosixPath('../blame/drivers/uio/uio_mf624.c.blame'): {'mf624_disable_interrupt': ['iowrite32',
   'ioread32',
   'iowrite32',
   'ioread32',
   'iowrite32',
   'ioread32'],
  'mf624_enable_interrupt': ['iowrite32',
   'ioread32',
   'iowrite32',
   'ioread32',
   'iowrite32',
   'ioread32'],
  'mf624_irq_handler': ['ioread32',
   'ioread32',
   'mf624_disable_interrupt',
   'ioread32',
   'ioread32',
   'mf624_disable_interrupt'],
  'mf624_irqcontrol': ['mf624_disable_interrupt', 'mf624_enable_interrupt'],
  'mf624_setup_mem': ['pci_resource_start',
   'pci_resource_len',
   'pci_ioremap_bar'],
  'mf624_pci_probe': ['kzalloc',
   'pci_enable_device',
   'pci_request_regions',
   'mf624_setup_mem',
   'mf624_setup_mem',
   'mf624_setup_mem',
   'uio_register_device',
   'pci_set_drvdata',
   'iounmap',
   'iounmap',
   'iounmap',
   'pci_release_regions',
   'pci_disable_device',
   'kfree'],
  'mf624_pci_remove': ['pci_get_drvdata',
   'mf624_disable_interrupt',
   'uio_unregister_dev

In [37]:
len(kernel_files_2)

29473

In [47]:
len([func for file in kernel_files_2.values() for func in file]) #529075 same number of functions found

529075

In [38]:
def reset_cregit_db():
    # all functions
    cursor.execute("CREATE TABLE IF NOT EXISTS cregit_functions (file, name)")
    # all calls from one function to annother
    cursor.execute("CREATE TABLE IF NOT EXISTS cregit_calls (file, caller, callee)")
    # clear tables
    cursor.execute("DELETE FROM cregit_functions")
    cursor.execute("DELETE FROM cregit_calls")

In [39]:
kernel_files = parse_kernel()
reset_cregit_db()
# trim ../blame/ and .blame from ends of path so that paths can be compared between methods
cursor.executemany("INSERT INTO cregit_calls VALUES (?,?,?)", (
    (str(file)[len("../blame/"):-len(".blame")], caller, callee)
    for file, functions in kernel_files.items()
    for caller, callees in functions.items()
    for callee in set(callees))
)
cursor.executemany("INSERT INTO cregit_functions VALUES (?,?)", (
    (str(file)[len("../blame/"):-len(".blame")], function_name)
    for file, file_functions in kernel_files.items()
    for function_name in file_functions
))

In [40]:
# get test functions (based on name, case insensitive)
print(head(cursor.execute("SELECT caller FROM cregit_calls WHERE caller LIKE '%test%'")))
# get tested functions
head(cursor.execute("SELECT COUNT(DISTINCT callee) FROM cregit_calls WHERE caller LIKE '%test%'"))

[('test_8',), ('test_8',), ('test_16',), ('test_16',), ('test_32',), ('test_32',), ('tx4939ide_dma_test_irq',), ('tx4939ide_dma_test_irq',), ('tx4939ide_dma_test_irq',), ('tx4939ide_dma_test_irq',)]


[(7480,)]

In [41]:
tested_functions = set(x[0] for x in cursor.execute("SELECT callee FROM cregit_calls WHERE caller LIKE '%test%'"))

In [42]:
# Total number of functions
print(head(cursor.execute("SELECT COUNT(*) FROM cregit_functions")))

[(529075,)]


In [43]:
print(head(cursor.execute("SELECT COUNT(DISTINCT name)*1.0/COUNT(*) FROM cregit_functions")))

[(0.9484817842460899,)]


In [44]:
list(cursor.execute(
    "WITH A AS (SELECT name FROM cregit_functions GROUP BY name HAVING COUNT(name)>3) SELECT COUNT(*) FROM A"))

[(1933,)]

In [45]:
list(cursor.execute(
    "SELECT name, COUNT(name) FROM cregit_functions GROUP BY name ORDER BY COUNT(name) DESC LIMIT 20"))

[('main', 717),
 ('usage', 85),
 ('f', 71),
 ('name_show', 61),
 ('uni2char', 52),
 ('char2uni', 52),
 ('sd_probe', 49),
 ('sd_init', 47),
 ('sd_config', 46),
 ('vidioc_querycap', 45),
 ('sd_start', 44),
 ('modalias_show', 42),
 ('platform_init', 41),
 ('sd_pkt_scan', 40),
 ('temp_show', 39),
 ('sd_init_controls', 38),
 ('plat_mem_setup', 38),
 ('to_state', 37),
 ('sd_stopN', 36),
 ('sd_s_ctrl', 36)]

In [ ]:
connection.commit()

# Comparing Ctags and Cregit

In [82]:
[(table_name, [x[1] for x in cursor.execute("PRAGMA table_info([%s])" % table_name)])
 for table_name in ("ctags", "cregit_functions", "cregit_calls")]

[('ctags', ['file', 'name', 'line', 'token_type', 'text']),
 ('cregit_functions', ['file', 'name']),
 ('cregit_calls', ['file', 'caller', 'callee'])]

In [87]:
head(cursor.execute("SELECT COUNT(*) FROM ctags WHERE token_type='function'")) # includes variables, macros etc.

[(590237,)]

In [84]:
head(cursor.execute("SELECT COUNT(*) FROM cregit_functions"))

[(529075,)]

In [85]:
head(cursor.execute("""
SELECT COUNT(*) FROM ctags JOIN cregit_functions ON
ctags.file = cregit_functions.file
AND ctags.name = cregit_functions.name
"""))

[(572355,)]

In [86]:
head(cursor.execute("""
SELECT file, name FROM cregit_functions
EXCEPT SELECT file, name FROM ctags"""))

[('arch/arm/mach-omap2/powerdomain.c', 'pwrdm_lock'),
 ('arch/powerpc/kernel/irq.c', 'decrementer_check_overflow'),
 ('arch/riscv/kernel/ftrace.c', 'ftrace_arch_code_modify_prepare'),
 ('arch/s390/kvm/vsie.c', 'do_vsie_run'),
 ('arch/um/drivers/random.c', 'rng_dev_open'),
 ('arch/um/drivers/ubd_kern.c', 'queue_rw_req'),
 ('arch/um/drivers/ubd_kern.c', 'ubd_queue_one_vec'),
 ('arch/x86/kernel/cpu/mtrr/generic.c', 'post_set'),
 ('arch/x86/kernel/cpu/mtrr/generic.c', 'prepare_set'),
 ('arch/x86/kernel/cpu/resctrl/rdtgroup.c', 'move_myself')]

In [90]:
head(cursor.execute(
    "SELECT file, name FROM cregit_functions"))

[('drivers/uio/uio_mf624.c', 'mf624_disable_interrupt'),
 ('drivers/uio/uio_mf624.c', 'mf624_enable_interrupt'),
 ('drivers/uio/uio_mf624.c', 'mf624_irq_handler'),
 ('drivers/uio/uio_mf624.c', 'mf624_irqcontrol'),
 ('drivers/uio/uio_mf624.c', 'mf624_setup_mem'),
 ('drivers/uio/uio_mf624.c', 'mf624_pci_probe'),
 ('drivers/uio/uio_mf624.c', 'mf624_pci_remove'),
 ('drivers/uio/uio_hv_generic.c', 'hv_uio_irqcontrol'),
 ('drivers/uio/uio_hv_generic.c', 'hv_uio_channel_cb'),
 ('drivers/uio/uio_hv_generic.c', 'hv_uio_rescind')]

In [92]:
head(cursor.execute(
    "SELECT file, name FROM ctags EXCEPT SELECT file, name FROM cregit_functions"))

[('Documentation/scheduler/sched-pelt.c', 'HALFLIFE'),
 ('Documentation/scheduler/sched-pelt.c', 'SHIFT'),
 ('Documentation/scheduler/sched-pelt.c', 'max'),
 ('Documentation/scheduler/sched-pelt.c', 'n'),
 ('Documentation/scheduler/sched-pelt.c', 'sum'),
 ('Documentation/scheduler/sched-pelt.c', 'y'),
 ('Documentation/usb/usbdevfs-drop-permissions.c',
  'USBDEVFS_CAP_DROP_PRIVILEGES'),
 ('Documentation/usb/usbdevfs-drop-permissions.c', 'USBDEVFS_DROP_PRIVILEGES'),
 ('arch/alpha/boot/bootp.c', 'KERNEL_ORIGIN'),
 ('arch/alpha/boot/bootp.c', 'L1')]

In [93]:
head(cursor.execute(
    "SELECT file, name FROM cregit_functions EXCEPT SELECT file, name FROM ctags"))

[('arch/arm/mach-omap2/powerdomain.c', 'pwrdm_lock'),
 ('arch/powerpc/kernel/irq.c', 'decrementer_check_overflow'),
 ('arch/riscv/kernel/ftrace.c', 'ftrace_arch_code_modify_prepare'),
 ('arch/s390/kvm/vsie.c', 'do_vsie_run'),
 ('arch/um/drivers/random.c', 'rng_dev_open'),
 ('arch/um/drivers/ubd_kern.c', 'queue_rw_req'),
 ('arch/um/drivers/ubd_kern.c', 'ubd_queue_one_vec'),
 ('arch/x86/kernel/cpu/mtrr/generic.c', 'post_set'),
 ('arch/x86/kernel/cpu/mtrr/generic.c', 'prepare_set'),
 ('arch/x86/kernel/cpu/resctrl/rdtgroup.c', 'move_myself')]

## Cscope

In [39]:
def parse_cscope(filename):
    expression = r"(?P<path>^[^ ]+) (?P<funcname>[^ ]+) (?P<linenum>\d+) (?P<usage_line>.*)"
    return parse(filename, expression)

In [14]:
# get all function calls
!cscope -RL2 ".*" > {all_calls}
# get all calls from a function with test in the name
!cscope -RL2 ".*test.*" > {test_targets}

In [15]:
called_functions = set()
called_function_names = set()
for m in parse_cscope(all_calls):
    called_functions.add((m["funcname"], m["path"]))
    called_function_names.add(m["funcname"])

In [16]:
tested_functions = set()
tested_function_names = set()     
for m in parse_cscope(test_targets):
    tested_functions.add((m["funcname"], m["path"]))
    tested_function_names.add(m["funcname"])

In [17]:
len(tested_function_names - called_function_names), tested_function_names - called_function_names
# before the math.c fix, there were 187 functions in this set

(184,
 {'PCH_PCR_GPIO_ADDRESS',
  'PPCSTRUCT',
  'QLC_83XX_CURRENT_LINK_SPEED',
  'QLC_83XX_LINK_STATE',
  'QLC_83xx_FUNC_VAL',
  'SSIP_PAYLOAD',
  'TestClearPageYoung',
  'VM86_ARG',
  'VM86_TYPE',
  '__comedi_get_user_chanlist',
  '__comedi_get_user_cmd',
  '__inorder_to_tree',
  '__test_and_change_bit',
  '__to_inorder',
  '_rng_recvmsg',
  'advance_polynomial',
  'allocate_frame_data',
  'an_exception',
  'blinkm_test_run',
  'bnxt_half_close_nic',
  'bnxt_half_open_nic',
  'bnxt_hwrm_mac_loopback',
  'bnxt_hwrm_phy_loopback',
  'bnxt_hwrm_selftest_irq',
  'bnxt_run_fw_tests',
  'bnxt_run_loopback',
  'bnxt_test_irq',
  'branch',
  'chacha20poly1305_encrypt_bignonce',
  'chacha20poly1305_selftest_encrypt',
  'check_execveat',
  'check_execveat_fail',
  'check_execveat_pathmax',
  'child_join_close',
  'concat',
  'config_copy_test_driver_name',
  'config_copy_test_fs',
  'copy_user_test',
  'cpcap_battery_get_state',
  'das16m1_ai_check_chanlist',
  'deallocate_frame_data',
  'debu

In [9]:
# functions detected by cscope (getting called by the tests) but not by ctags
len(tested_function_names - func_names), len(tested_functions - functions)

NameError: name 'func_names' is not defined

In [ ]:
target = KERNEL

!(cd {target}; cscope -RL0 ".*")

In [12]:
# the results of this cell seem to indicate that a single directory (arch/sh/math-emu) is tripping up cscope.
# removeing that file lets cscope run unimpeded,
def test_cscope(path):
    if not path.is_dir():
        print("fail on file:",path)
        return
    for p in path.iterdir():
        if p.is_dir():
            print(p)
            result = run('cscope -RL0 ".*"',cwd=p, capture_output=True, shell=True)
            # return code is a better way to identify if errors
            # could just list all dirs/files in kernel, and echo stderr to each
            # find -d (gives list of directories)
            if result.stderr not in {b'', b'cscope: no source files found\n'}:
                print(p, repr(result.stderr))
                test_cscope(p)

test_cscope(KERNEL)

.ipynb_checkpoints
drivers
sound
samples
virt
LICENSES
arch
certs
lib
mm
block
scripts
net
include
security
kernel
tools
crypto
usr
Documentation
fs
init
ipc


In [ ]:
result = run('cscope -RL0 ".*"', capture_output=True, shell=True)
result